In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
%matplotlib inline
import cv2
import os
import json
from tqdm import tqdm

In [2]:
PROJECT_PATH = '...'
DATA_PATH = '...'
MODEL_PATH = '...'
SUBMIT_PATH = PROJECT_PATH + 'submission_files/'

In [3]:
from ultralytics import YOLO

model = YOLO(MODEL_PATH + 'yolon_aug_eq_e150_sgd_best.pt')

In [ ]:
import time
start = time.time()
results = model.predict(DATA_PATH, save = False, conf = 0.65, device = 1)
print((time.time() - start) / 60)

In [5]:
img_id_dict = dict()

with open(PROJECT_PATH + 'test.json', "r") as f:
    label_coco = json.load(f)

for val in label_coco['images']:
    img_id_dict[val['file_name']] = val['id']

img_id_dict['test_0.png']

0

In [6]:
# submission 파일 만들기
def make_submission_yolo(result, file_name):
  output = result.copy()
  with open(PROJECT_PATH + 'test.json', 'r') as f:
      lab = json.load(f)
    
  # 객체가 탐지된 이미지
  cnt = 0
  for pred in output:
      if len(pred.boxes) != 0:
          cnt +=1
  print('탐지된 파일 수: ',cnt)

  annot = []

  for pred in tqdm(output):
      if len(pred.boxes) != 0: # 객체가 존재할 때만
        for i in range(len(pred.boxes)):
            # if pred.boxes.xyxy[i].tolist()[0] > 0 and pred.boxes.xyxy[i].tolist()[1] > 0 and \
            # pred.boxes.xyxy[i].tolist()[0] < 640 and pred.boxes.xyxy[i].tolist()[1] < 480: # box가 이미지 범위를 넘어가면(물고기가 사진이 잘린 것)
              result_dict = dict()
              result_dict['id'] = len(annot)
              result_dict['image_id'] = img_id_dict[pred.path.split('/')[-1]]
              result_dict['category_id'] = int(pred.boxes.cls.tolist()[i])

              #bbox
              x = pred.boxes.xyxy[i].tolist()[0] # 여기 있는 x가 x0
              y = pred.boxes.xyxy[i].tolist()[1] # y0
              w = pred.boxes.xywh[i].tolist()[2] # w
              h = pred.boxes.xywh[i].tolist()[3] # h

              result_dict['bbox'] = [x,y,w,h]
              result_dict['area'] = w * h
              annot.append(result_dict)
  
  lab['annotations'] = annot
  print('탐지된 객체 수: ', len(lab['annotations']))
  print(lab['annotations'][:3])
  cat_list = [x['category_id'] for x in lab['annotations']]
  display(pd.Series(cat_list).value_counts().sort_values(ascending = False))

  with open(f'{SUBMIT_PATH}{file_name}', "w") as json_file:
    json.dump(lab, json_file, ensure_ascii=False, indent=4)

  return lab['annotations']

In [7]:
ann = make_submission_yolo(results, 'yolon_aug_eq_e150_sgd_c65.json')

탐지된 파일 수:  3458


100%|████████████████████████████████████████████████████████████████████████████| 44946/44946 [00:00<00:00, 48777.22it/s]

탐지된 객체 수:  3691
[{'id': 0, 'image_id': 0, 'category_id': 3, 'bbox': [108.0645751953125, 215.0891571044922, 351.44598388671875, 102.18836975097656], 'area': 35913.69214891177}, {'id': 1, 'image_id': 2, 'category_id': 3, 'bbox': [0.98358154296875, 218.93084716796875, 361.0084228515625, 112.9547119140625], 'area': 40777.6024017483}, {'id': 2, 'image_id': 293, 'category_id': 5, 'bbox': [90.670654296875, 266.48199462890625, 488.3497314453125, 135.1318359375], 'area': 65991.59578979015}]


1    1410
3    1181
2     368
7     361
0     113
5     112
4      84
6      62
dtype: int64